In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
import numpy 
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import pyarrow
import sys

from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path
from typing import List,Dict, Tuple
from collections import defaultdict
pd.set_option("display.max_columns", None)

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts")))

import create_queries
import prepare_data
from tqdm import tqdm

from bespoke.inventory.analysis.shared import download_util, inventory_types
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util

%load_ext autoreload
%autoreload 2

In [3]:
from underwriting import client_surveillance_jupyter
from typing import Union,cast


In [4]:
TRANSFER_PACKAGES_START_DATE = '2021-01-01'
SALES_TRANSACTIONS_START_DATE = '2021-01-01'

In [5]:
ca_df = pd.read_csv('dec_test.csv',index_col = 0)
ca_df.columns = ['company_name','company_identifier','state','licenses']
ca_df['company_identifier'] = ca_df['company_identifier'].astype(str)

In [6]:
ca_df

,company_name,company_identifier,state,licenses
0,64 and Hope Melrose,64HM,CA,C10-0000922-LIC
1,99 High Tide,99HT,CA,C10-0000279-LIC
2,A Green Alternative Inc.,AGA,CA,C10-0000341-LIC
3,Altum LLC,ALT,MI,PC-000310;AU-R-000380
4,Boston Bud Factory,BBF,MA,MR281525
...,...,...,...,...
110,Urbn Leaf,UR,CA,C10-0000776-LIC;C10-0000246-LIC;C10-0000731-LI...
111,Voyage,VOY,CA,C12-0000030-LIC;C10-0000802-LIC;C12-0000159-LIC
112,Verts,VS,CO,402-00251;402R-00545
113,Palm Desert Melanie Place Industrial,WCCC,CA,C10-0000045-LIC;C10-0000372-LIC;C12-0000119-LIC


In [7]:
ca_df_clean = ca_df[~ca_df['company_identifier'].isin(['WHT','GF','UR'])]
ca_df_clean.shape

(112, 4)

In [8]:
ca_df_clean['lic_list'] = [list(ca_df_clean['licenses'])[i].split(";") for i in range(len(ca_df_clean))]

In [9]:
ca_df_clean = ca_df_clean[ca_df_clean['state'] == 'CA']

In [10]:
ca_df_clean['company_identifier'].nunique()

78

In [11]:
ca_df['company_name'].unique()

array(['64 and Hope Melrose', '99 High Tide', 'A Green Alternative Inc.',
       'Altum LLC', 'Boston Bud Factory', "Bud's",
       'Big Medicine Cannabissary', 'Buds & Roses Collective Inc',
       'Bella Sol', 'Budee', 'Bynate', 'Catalyst', 'Calm and Collective',
       "California's Choice Collective", 'Chad Enterprises',
       'CanEx Delivery', 'Cali Greens LLC', 'Colorado Holdings LLC',
       'CHO', 'Calilove Connection', 'California Patients Alliance',
       'California Patients Club', 'Cana Rose Realty Holdings, LLC',
       'California Street Cannabis',
       'California Street Cannabis at Clement', 'Diem Cannabis OR',
       'Dunegrass', 'Dubs Green Garden', 'Diem Lynn', 'TDMA',
       'Dalwhinnie Farms', 'Eskar Holdings', 'E-Leaf', 'Embarc Alameda',
       'Embarc Blackstone', 'Embarc Fairfield', 'Embarc Martinez',
       'Embarc Sacramento', 'Embarc Tahoe',
       'Emerald Perspective Cooperative, Inc', 'Fresh Baked CO',
       'The Flore Store', 'Falcon', 'Futureworks',

In [12]:
ca_df_clean_ex = ca_df_clean.explode('lic_list')

In [13]:
ca_df_clean_ex = ca_df_clean_ex[~ca_df_clean_ex['company_name'].isin(['Pure Roots, LLC '])]

In [14]:
ca_df_clean_ex[ca_df_clean_ex['company_identifier'] == 'PRL']

,company_name,company_identifier,state,licenses,lic_list
86,People's Remedy Lone Palm,PRL,CA,PC-000381;PC-000749;PC-000803;AU-R-000229;AU-R...,PC-000381
86,People's Remedy Lone Palm,PRL,CA,PC-000381;PC-000749;PC-000803;AU-R-000229;AU-R...,PC-000749
86,People's Remedy Lone Palm,PRL,CA,PC-000381;PC-000749;PC-000803;AU-R-000229;AU-R...,PC-000803
86,People's Remedy Lone Palm,PRL,CA,PC-000381;PC-000749;PC-000803;AU-R-000229;AU-R...,AU-R-000229
86,People's Remedy Lone Palm,PRL,CA,PC-000381;PC-000749;PC-000803;AU-R-000229;AU-R...,AU-R-000607
86,People's Remedy Lone Palm,PRL,CA,PC-000381;PC-000749;PC-000803;AU-R-000229;AU-R...,AU-R-000739
86,People's Remedy Lone Palm,PRL,CA,PC-000381;PC-000749;PC-000803;AU-R-000229;AU-R...,PC-000814
86,People's Remedy Lone Palm,PRL,CA,PC-000381;PC-000749;PC-000803;AU-R-000229;AU-R...,C10-0000759-LIC


In [15]:
def get_identifiers_for_where_clause(company_identifier
):

    if type(company_identifier) == str:
        identifiers.append(cast(str, company_identifier))
    else:
        identifiers = cast(List[str], company_identifier)

    return ['"{}"'.format(iden) for iden in identifiers]

In [61]:
def create_query(LICENSE_NUMBER_LIST):
    return f"""
with incoming as (
    select
  COALESCE(
    company_licenses.company_id, incoming.shipper_facility_license_number
  ) shipper_company_id,
  COALESCE(
    companies.parent_company_id, incoming.shipper_facility_license_number
  ) shipper_parent_company_id,
  COALESCE(
    parent_companies.name, incoming.shipper_facility_name
  ) shipper_parent_name,
  incoming.created_date,
  incoming.received_datetime,
  incoming.shipper_facility_name,
  incoming.recipient_facility_name,
  incoming.package_id,
  incoming.type,
  incoming.product_name,
  incoming.shipper_wholesale_price,
  incoming.shipped_quantity,
  incoming.receiver_wholesale_price,
  incoming.updated_at,
  incoming.recipient_facility_license_number
from
  dbt_transformation.int__company_incoming_transfer_packages incoming
  left join dbt_transformation.stg__company_licenses company_licenses on incoming.shipper_facility_license_number = company_licenses.license_number
  left join dbt_transformation.stg__companies companies on company_licenses.company_id = companies.id
  left join dbt_transformation.stg__parent_companies parent_companies on companies.parent_company_id = parent_companies.id
WHERE
    incoming.recipient_facility_license_number IN ({','.join(LICENSE_NUMBER_LIST)})
    AND DATE(incoming.created_date) >= DATE('2021-01-01')
    AND DATE(incoming.created_date) < DATE('2023-11-01')
    ),
    
    sales as (
    select
    license_number,
    count(distinct receipt_number) count_tran
    from (
             select metrc_sales_receipts.license_number,
                    metrc_sales_receipts.receipt_number,
                    metrc_sales_receipts.type                              as rt_type,
                    metrc_sales_receipts.sales_customer_type,
                    metrc_sales_receipts.sales_datetime,
                    date_trunc(metrc_sales_receipts.sales_datetime, month) as sales_month,
                    metrc_sales_receipts.total_packages,
                    metrc_sales_receipts.total_price                       as rt_total_price,
                    metrc_sales_transactions.type                          as tx_type,
                    metrc_sales_transactions.package_id                    as tx_package_id,
                    metrc_sales_transactions.package_label                 as tx_package_label,
                    metrc_sales_transactions.product_name                  as tx_product_name,
                    metrc_sales_transactions.product_category_name         as tx_product_category_name,
                    metrc_sales_transactions.unit_of_measure               as tx_unit_of_measure,
                    metrc_sales_transactions.quantity_sold                 as tx_quantity_sold,
                    metrc_sales_transactions.total_price                   as tx_total_price
             from `bespoke-financial`.ProdMetrcData.metrc_sales_receipts metrc_sales_receipts
                      inner join `bespoke-financial`.ProdMetrcData.companies companies
                                 on metrc_sales_receipts.company_id = companies.id
                      left outer join `bespoke-financial`.ProdMetrcData.metrc_sales_transactions metrc_sales_transactions
                                      on metrc_sales_receipts.id = metrc_sales_transactions.receipt_row_id
             where metrc_sales_receipts.license_number in ({','.join(LICENSE_NUMBER_LIST)})
               and metrc_sales_receipts.sales_datetime >= "2022-11-01"
               and metrc_sales_receipts.sales_datetime <= "2022-12-31"
         )
    group by 1
)
select incoming.*,
       sales.count_tran
       from incoming left join sales on incoming.recipient_facility_license_number = sales.license_number
       where sales.count_tran > 0
    
    """

In [62]:
def read_df_in(license_numbers):
    company_incoming_transfer_packages_query = create_query(
    LICENSE_NUMBER_LIST=license_numbers,
    )
    company_incoming_transfer_packages_dataframe = pd.read_sql_query(company_incoming_transfer_packages_query, engine)
    return company_incoming_transfer_packages_dataframe


In [63]:
WINDOW = 2
VC_START_DATE = '2021-01-01'
VC_END_DATE = '2022-12-01'
#VC_MONTH_LIST = ['2022-08-01','2022-09-01','2022-10-01','2022-11-01']
VC_MONTH_LIST = ['2022-11-01','2022-12-01']

VC_MONTH_END = VC_MONTH_LIST[-1]

In [64]:
def get_vc_list(company_list,transfer_packages_start_date,sales_transactrions_start_date,map_df):
    vc_df_list = {}
    for c in tqdm(company_list):
        vc_list = {}
        print(c)
        l_list = map_df[map_df['company_identifier'] == c]['lic_list'].to_list()[0]
        for l in tqdm(l_list):
            print(l)
            df_in = read_df_in(get_identifiers_for_where_clause([l]))
            if df_in.shape[0] == 0:
                vc_list[l] = numpy.nan
                continue
            else:
                vc = client_surveillance_jupyter.calculate_vendor_churn_short(df_in,[l],VC_START_DATE,VC_END_DATE,VC_MONTH_LIST,VC_MONTH_END)[1]
                vc_list[l] = vc
        vc_df_list[c] =  vc_list
    return vc_df_list


In [65]:
#calculate VC with 2m window
c_2 = get_vc_list(list(ca_df_clean['company_identifier']),TRANSFER_PACKAGES_START_DATE,\
                SALES_TRANSACTIONS_START_DATE,ca_df_clean)

  0%|          | 0/79 [00:00<?, ?it/s]

64HM



  0%|          | 0/1 [00:00<?, ?it/s]

C10-0000922-LIC



  1%|▏         | 1/79 [00:05<07:38,  5.88s/it]

...


 33%|███▎      | 1/3 [00:05<00:10,  5.12s/it]

C10-0000372-LIC



 67%|██████▋   | 2/3 [00:10<00:05,  5.34s/it]

C12-0000119-LIC



100%|██████████| 79/79 [11:50<00:00,  9.00s/it]


In [66]:
df_vc_2 = pd.DataFrame()
for k in c_2.keys():
    print(k)
    for l in c_2[k].keys():
        print(l)
        data_vc_2 = c_2[k][l]
        if str(data_vc_2) == 'nan':
            continue
        else:
            data_vc_2['company'] = k
            data_vc_2['location'] = l
            print(data_vc_2.shape)
            df_vc_2 = df_vc_2.append(data_vc_2)

64HM
C10-0000922-LIC
(97, 34)
99HT
C10-0000279-LIC
(150, 34)
AGA
C10-0000341-LIC
(109, 34)
BRC
C10-0000603-LIC
BUD
C9-0000464-LIC
C9-0000167-LIC
C9-0000399-LIC
C9-0000467-LIC
C9-0000444-LIC
BYN
C10-0000528-LIC
(91, 34)
CAT
C10-0000364-LIC
(122, 34)
C10-0000885-LIC
(92, 34)
C10-0001165-LIC
(31, 34)
C10-0001177-LIC
C10-0000227-LIC
(120, 34)
C10-0000230-LIC
(159, 34)
C10-0000525-LIC
(126, 34)
C10-0000801-LIC
(114, 34)
C10-0000865-LIC
(92, 34)
C10-0000903-LIC
(104, 34)
C10-0000915-LIC
(61, 34)
C10-0000937-LIC
(68, 34)
C10-0001020-LIC
(50, 34)
C10-0001021-LIC
(57, 34)
C10-0001125-LIC
CC
C10-0000758-LIC
(109, 34)
CCC
C12-0000087-LIC
(107, 34)
CE
C12-0000334-LIC
(45, 34)
CED
C9-0000094-LIC
CG
C9-0000157-LIC
CLC
C10-0000986-LIC
(22, 34)
C12-0000318-LIC
CPA
C9-0000348-LIC
(52, 34)
C10-0000461-LIC
(108, 34)
CPC
C9-0000056-LIC
CRRH
C12-0000308-LIC
C12-0000405-LIC
(60, 34)
CSC
C10-0000670-LIC
(137, 34)
CSCC
C10-0000918-LIC
(65, 34)
DGG
C9-0000016-LIC
(53, 34)
EL
C9-0000370-LIC
(30, 34)
EMA
C10-000

In [67]:
df_vc_2[(df_vc_2['location'] == 'C10-0001096-LIC')]


year_month,shipper_parent_name,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,2022-11-01,2022-12-01,facility_total,grand_total,perc_total,last_4m_total,last_4m_active,significant,measure,company,location
0,420-3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,10198.47,0.00,0.00,0.00,0.00,10198.47,4414083.21,0.002310,0.00,False,True,Inactive,EMB,C10-0001096-LIC
1,965 Solutions,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5228.24,2498.53,6161.78,0.00,0.00,0.00,13888.55,4414083.21,0.003146,0.00,False,True,Inactive,EMB,C10-0001096-LIC
2,"ASV EQ 1, LLC Dba 710 Labs",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,16384.81,21538.12,20455.64,0.00,25084.04,83462.61,4414083.21,0.018908,25084.04,True,True,Active,EMB,C10-0001096-LIC
3,Aureus DHS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4414083.21,0.000000,0.00,False,False,Exclude,EMB,C10-0001096-LIC
4,Bango Distribution,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,13192.75,6784.36,0.00,1750.02,1727.07,23454.20,4414083.21,0.005313,3477.09,True,True,Active,EMB,C10-0001096-LIC
5,Bloom Network,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4414083.21,0.000000,0.00,False,False,Exclude,EMB,C10-0001096-LIC
6,Cabo Connection,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1296.48,2162.16,3133.68,0.00,0.00,0.00,6592.32,4414083.21,0.001493,0.00,False,True,Inactive,EMB,C10-0001096-LIC
7,"California Organic Treatment Center, Inc",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26574.92,39945.66,7339.30,8833.31,30973.70,0.00,113666.89,4414083.21,0.025751,30973.70,True,True,Active,EMB,C10-0001096-LIC
8,Claybourne Industries,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4255.35,9811.72,0.00,16052.77,8142.85,13785.00,52047.69,4414083.21,0.011791,21927.85,True,True,Active,EMB,C10-0001096-LIC
9,Connected Cannabis Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,47582.36,108070.59,40585.25,59602.58,26099.08,47631.95,329571.81,4414083.21,0.074664,73731.03,True,True,Active,EMB,C10-0001096-LIC


In [68]:
df_vc_2[(df_vc_2['location'] == 'C10-0001096-LIC')&\
        (df_vc_2['measure'] == 'Inactive')&\
        (df_vc_2['shipper_parent_name'].isin(list(tier_a_vendor)))][['shipper_parent_name','perc_total']]                                                                                     
                                                                                       
                                                                                       

year_month,shipper_parent_name,perc_total
20,Greenfield Organix dba Loudpack,0.013818
24,Irrational Raccoon,0.001426
48,Wonderbread,0.002444


In [ ]:
# num_inactive_2m_list = {}
# num_inactive_4m_list = {}
# for c in df_vc_state_2['location'].unique():
#     vc_c = df_vc_state_2[df_vc_state_2['location'] == c]
#     num_inactive_2m = vc_c[vc_c['measure'] == 'Inactive'].shape[0]
#     num_inactive_4m = vc_c[(vc_c['measure'] == 'Inactive')&(vc_c['inactive_4m'] == False)].shape[0]
#     num_inactive_2m_list[c] = num_inactive_2m
#     num_inactive_4m_list[c] = num_inactive_4m
    

In [ ]:
# num_inactive_2m_df = pd.DataFrame.from_dict(num_inactive_2m_list,orient ='index').reset_index()
# num_inactive_4m_df = pd.DataFrame.from_dict(num_inactive_4m_list,orient ='index').reset_index()

# #merge
# num_inactive_backtest_df = num_inactive_2m_df.merge(num_inactive_4m_df,how = 'left',on = 'index')

# # col names
# num_inactive_backtest_df.columns = ['license','num_churn_2m','num_not_churn_4m']
# # join with id
# num_inactive_backtest_df = num_inactive_backtest_df.merge(ca_df_clean_ex[['lic_list','company_identifier']],\
#                                      how = 'left',left_on = 'license',right_on = 'lic_list')

In [ ]:
#num_inactive_backtest_df.to_csv('num_inactive_backtest_df_dec.csv')


# tiering analysis

In [69]:
tier_a_vendor = set(['Herbl',
'River Distribution (Continuum)',
'Dreamfields',
'Kiva Sales & Service',
'Mammoth Distro (Heavy Hitters)',
'Artisan Canna Cigars',
'NW Confections CA',
'Four Star Manufacturing',
'Kind House',
'Falcon',
'Cypress Manufacturing Co',
'Irrational Raccoon',
'Nabis',
'Ironworks Collective',
'Claybourne Industries',
'Umbrla',
'Catalyst',
'Grizzly Peak Farms',
'Growpacker',
'1346 LA LLC',
'Papa and Barkley',
'Mvn productions',
'Wonderbread',
'New Age Care Center',
'NorCal Distribution Solutions',
'Greenfield Organix dba Loudpack',
'HGDCO Inc',
'Green Dragon Caregivers',
'Cali Flwr',
'Perfect Union'])

In [ ]:
# tier_b_vendor = set(['New Age Care Center',
# 'NorCal Distribution Solutions',
# 'Greenfield Organix dba Loudpack',
# 'HGDCO Inc',
# 'PRUSSIAN STICKS LLC',
# 'Mammoth Distro (Heavy Hitters)',
# 'MDS BUSINESS SERVICES LLC',
# 'Perfect Union',
# 'PRUSSIAN STICKS LLC',
# 'UpNorth Humboldt',
# 'Connected Cannabis Co',
# 'Standard Holdings',
# 'Hypeereon Corporation',
# 'Herbarium',
# 'Big Tree Industries',
# 'Wonderbread',
# 'Cypress Manufacturing Co',
# 'HERBAL REMEDIES CAREGIVERS INC.',
# 'Good Tree',
# 'Ten Point Marketing LLC'])

In [ ]:
# tier_c_vendor = set(['E&J Distributors',
# 'VARAVO',
# 'Coastal Distribution',
# 'Melrose Facility',
# 'Palm Desert Melanie Place Industrial',
# 'ASV EQ 1 LLC Dba 710 Labs',
# 'Aureus DHS',
# 'WCC MGMT',
# 'Pacific West Distributors',
# 'Liquid Shade'])

In [70]:
churn_a_list_2 = {}
#churn_b_list_2 = {}
#churn_c_list_2 = {}
total_churn_list_2 = {}
purchase_a_list_2 = {}
#purchase_b_list_2 = {}
#purchase_c_list_2 = {}
total_purchase_list_2 = {}

for c in df_vc_2['location'].unique():#df_vc_state_2['location'].unique():
    vc_c_2 = df_vc_2[df_vc_2['location'] == c]
    i = vc_c_2['company'].unique()
    print(i)
    #churn
    total_churn_2 = vc_c_2[vc_c_2['measure'] == 'Inactive']['perc_total'].sum()
    total_churn_list_2[c] = total_churn_2
    #purchase
    total_purchase_2 = vc_c_2['last_4m_total'].sum()
    total_purchase_list_2[c] = total_purchase_2
    if len(set(vc_c_2['shipper_parent_name'].unique()).intersection(tier_a_vendor)) == 0:
        print(c)
        churn_df_a_2 = 0
        purchase_df_a_2 = 0
    else:
        #churn
        churn_df_a_2 = vc_c_2[(vc_c_2['shipper_parent_name'].isin(list(tier_a_vendor)))&\
                          (vc_c_2['measure'] == 'Inactive')]['perc_total'].sum()
#         churn_df_b_2 = vc_c_2[(vc_c_2['shipper_parent_name'].isin(list(tier_b_vendor)))&\
#                           (vc_c_2['measure'] == 'Inactive')]['perc_total'].sum()
#         churn_df_c_2 = vc_c_2[(vc_c_2['shipper_parent_name'].isin(list(tier_c_vendor)))&\
#                           (vc_c_2['measure'] == 'Inactive')]['perc_total'].sum()
        #purchase
        purchase_df_a_2 = vc_c_2[vc_c_2['shipper_parent_name'].isin(list(tier_a_vendor))]['last_4m_total'].sum()
#         purchase_df_b_2 = vc_c_2[vc_c_2['shipper_parent_name'].isin(list(tier_b_vendor))]['last_4m_total'].sum()
#         purchase_df_c_2 = vc_c_2[vc_c_2['shipper_parent_name'].isin(list(tier_c_vendor))]['last_4m_total'].sum()
        
    churn_a_list_2[c] = churn_df_a_2
#     churn_b_list_2[c] = churn_df_b_2
#     churn_c_list_2[c] = churn_df_c_2
    purchase_a_list_2[c] = purchase_df_a_2
#     purchase_b_list_2[c] = purchase_df_b_2
#     purchase_c_list_2[c] = purchase_df_c_2

['64HM']
['99HT']
['AGA']
['BYN']
['CAT']
['CAT']
['CAT']
['CAT']
['CAT']
['CAT']
['CAT']
['CAT']
['CAT']
['CAT']
['CAT']
['CAT']
['CAT']
['CC']
['CCC']
['CE']
['CLC']
['CPA']
['CPA']
['CRRH']
['CSC']
['CSCC']
['DGG']
['EL']
['EMA']
['EMB']
['EMF']
['EMM']
['EMS']
['EMT']
['EPC']
['FD']
['FI']
['FI']
['FI']
['GDN']
['GDS']
['GFEE']
C10-0000224-LIC
['GFWF']
['GHC']
['GL']
['GT']
C9-0000385-LIC
['GTC']
['HB']
['HB']
['HGD']
['HPCC']
['HPCC']
['HS']
['ID']
['IDC']
['LBC']
['MD']
['ML']
['MPW']
['MR']
['MSS']
['MW']
['NCDS']
['NCDS']
['PCC']
['PL']
['PL']
['PRL']
PC-000381
['PRL']
['PRL']
AU-R-000739
['PWD']
['RDR']
['RGD']
['SFV']
['SGF']
['SLCC']
['ST']
['T4L']
['TGL']
['TJR']
['TT']
['UHHC']
['UHHC']
['VOY']
['WCCC']
['WCCC']
['WCCC']


In [71]:
total_churn_df_2 = pd.DataFrame.from_dict(total_churn_list_2,orient ='index').reset_index()
churn_a_df_2 = pd.DataFrame.from_dict(churn_a_list_2,orient ='index').reset_index()
# churn_b_df_2 = pd.DataFrame.from_dict(churn_b_list_2,orient ='index').reset_index()
# churn_c_df_2 = pd.DataFrame.from_dict(churn_c_list_2,orient ='index').reset_index()

# merge
vc_backtest_df_2 = total_churn_df_2.merge(churn_a_df_2,how = 'left',on = 'index')\
# .merge(churn_b_df_2,how = 'left',on = 'index')\
# .merge(churn_c_df_2,how = 'left',on = 'index')

#column names
vc_backtest_df_2.columns = ['license','total','tier_a']
# join with id
vc_backtest_df_2 = vc_backtest_df_2.merge(ca_df_clean_ex[['lic_list','company_identifier']],\
                                      how = 'left',left_on = 'license',right_on = 'lic_list')

In [72]:
vc_backtest_df_2[vc_backtest_df_2['lic_list'] == 'C10-0000224-LIC']

,license,total,tier_a,lic_list,company_identifier
41,C10-0000224-LIC,0.013389,0.0,C10-0000224-LIC,GFEE


In [73]:
vc_backtest_df_2

,license,total,tier_a,lic_list,company_identifier
0,C10-0000922-LIC,0.145886,0.021852,C10-0000922-LIC,64HM
1,C10-0000279-LIC,0.313852,0.142034,C10-0000279-LIC,99HT
2,C10-0000341-LIC,0.189144,0.040679,C10-0000341-LIC,AGA
3,C10-0000528-LIC,0.639914,0.441150,C10-0000528-LIC,BYN
4,C10-0000364-LIC,0.250277,0.120235,C10-0000364-LIC,CAT
...,...,...,...,...,...
82,C9-0000082-LIC,1.000000,0.016034,C9-0000082-LIC,UHHC
83,C10-0000802-LIC,0.029385,0.009581,C10-0000802-LIC,VOY
84,C10-0000045-LIC,0.016528,0.009271,C10-0000045-LIC,WCCC
85,C10-0000372-LIC,0.000000,0.000000,C10-0000372-LIC,WCCC


In [74]:
vc_backtest_df_2['tier_a_perc'] = vc_backtest_df_2['tier_a'] / vc_backtest_df_2['total']
vc_backtest_df_2

,license,total,tier_a,lic_list,company_identifier,tier_a_perc
0,C10-0000922-LIC,0.145886,0.021852,C10-0000922-LIC,64HM,0.149787
1,C10-0000279-LIC,0.313852,0.142034,C10-0000279-LIC,99HT,0.452551
2,C10-0000341-LIC,0.189144,0.040679,C10-0000341-LIC,AGA,0.215068
3,C10-0000528-LIC,0.639914,0.441150,C10-0000528-LIC,BYN,0.689390
4,C10-0000364-LIC,0.250277,0.120235,C10-0000364-LIC,CAT,0.480410
...,...,...,...,...,...,...
82,C9-0000082-LIC,1.000000,0.016034,C9-0000082-LIC,UHHC,0.016034
83,C10-0000802-LIC,0.029385,0.009581,C10-0000802-LIC,VOY,0.326038
84,C10-0000045-LIC,0.016528,0.009271,C10-0000045-LIC,WCCC,0.560934
85,C10-0000372-LIC,0.000000,0.000000,C10-0000372-LIC,WCCC,NaN


In [75]:
total_purchase_df_2 = pd.DataFrame.from_dict(total_purchase_list_2,orient ='index').reset_index()
purchase_a_df_2 = pd.DataFrame.from_dict(purchase_a_list_2,orient ='index').reset_index()
#purchase_b_df_2 = pd.DataFrame.from_dict(purchase_b_list_2,orient ='index').reset_index()
#purchase_c_df_2 = pd.DataFrame.from_dict(purchase_c_list_2,orient ='index').reset_index()

# merge
vc_backtest_df_purchase_2 = total_purchase_df_2.merge(purchase_a_df_2,how = 'left',on = 'index')\
#.merge(purchase_b_df_2,how = 'left',on = 'index')\
#.merge(purchase_c_df_2,how = 'left',on = 'index')

# col names
vc_backtest_df_purchase_2.columns = ['license','total','tier_a']
# merge with id
vc_backtest_df_purchase_2 = vc_backtest_df_purchase_2.merge(ca_df_clean_ex[['lic_list','company_identifier']],\
                                      how = 'left',left_on = 'license',right_on = 'lic_list')

In [76]:
vc_backtest_df_purchase_2['tier_a_purchase_perc'] = vc_backtest_df_purchase_2['tier_a'] / vc_backtest_df_purchase_2['total']
vc_backtest_df_purchase_2

,license,total,tier_a,lic_list,company_identifier,tier_a_purchase_perc
0,C10-0000922-LIC,67926.19,26695.36,C10-0000922-LIC,64HM,0.393005
1,C10-0000279-LIC,125729.72,59669.49,C10-0000279-LIC,99HT,0.474585
2,C10-0000341-LIC,570995.95,235106.78,C10-0000341-LIC,AGA,0.411749
3,C10-0000528-LIC,31339.96,3415.33,C10-0000528-LIC,BYN,0.108977
4,C10-0000364-LIC,263719.87,199517.80,C10-0000364-LIC,CAT,0.756552
...,...,...,...,...,...,...
82,C9-0000082-LIC,0.00,0.00,C9-0000082-LIC,UHHC,NaN
83,C10-0000802-LIC,79045.09,0.00,C10-0000802-LIC,VOY,0.000000
84,C10-0000045-LIC,74737.34,0.00,C10-0000045-LIC,WCCC,0.000000
85,C10-0000372-LIC,35837.09,0.00,C10-0000372-LIC,WCCC,0.000000


In [77]:
vc_backtest_df_purchase_2[vc_backtest_df_purchase_2['license'] == 'PC-000381']

,license,total,tier_a,lic_list,company_identifier,tier_a_purchase_perc
67,PC-000381,2340.0,0.0,PC-000381,PRL,0.0


In [79]:
vc_backtest_df_2[vc_backtest_df_2['lic_list'] == 'C9-0000467-LIC']


,license,total,tier_a,lic_list,company_identifier,tier_a_perc


In [80]:
vc_backtest_df_2.to_csv('vcv2_2_01102023.csv')
vc_backtest_df_purchase_2.to_csv('vcv2_purchase_2_01102023.csv')